In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.2 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Separate features and labels
X = data.drop('Attrition', axis=1)
y = data['Attrition']

# Define categorical and numerical column
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# Create transformers for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Apply the transformation to the data
X_preprocessed = preprocessor.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Initialize the Random Forest classifier
rf = RandomForestClassifier()

# Define the parameters for the search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt', 'log2'],   # Number of features to consider at each split
    'max_depth': [None, 10, 20, 30],   # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],   # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] } # Minimum number of samples required to be at a leaf node
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:")
print(grid_search.best_params_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

Mejores parámetros encontrados:
{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Get the best model
best_rf = grid_search.best_estimator_

# Predict on the test set
y_pred = best_rf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8707482993197279
Classification Report:
               precision    recall  f1-score   support

          No       0.88      0.99      0.93       255
         Yes       0.57      0.10      0.17        39

    accuracy                           0.87       294
   macro avg       0.72      0.55      0.55       294
weighted avg       0.84      0.87      0.83       294



## LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Initialize the Logistic Regression classifier
log_reg = LogisticRegression(max_iter=1000)  # max_iter es importante para asegurar la convergencia

# Define the parameters for the search
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],  # Tipo de regularización
    'C': [0.1, 1, 10, 100],  # Inverso de la regularización
    'solver': ['liblinear', 'saga'],  # Algoritmos para resolver la regresión logística
    'class_weight': [None, 'balanced']  # Pesos para las clases desbalanceadas
}

# Set up the GridSearchCV

grid_search_log_reg = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model to the training data
grid_search_log_reg.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:")
print(grid_search_log_reg.best_params_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
Mejores parámetros encontrados:
{'C': 0.1, 'class_weight': None, 'penalty': 'l2', 'solver': 'liblinear'}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
120 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1169, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Get the best model
best_log_reg = grid_search_log_reg.best_estimator_

# Predict on the test set
y_pred_log_reg = best_log_reg.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

Accuracy: 0.9013605442176871
Classification Report:
               precision    recall  f1-score   support

          No       0.91      0.98      0.95       255
         Yes       0.78      0.36      0.49        39

    accuracy                           0.90       294
   macro avg       0.84      0.67      0.72       294
weighted avg       0.89      0.90      0.89       294



## CATBOOST

In [ ]:
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# Initialize CatBoost
catboost_model = CatBoostClassifier(learning_rate=0.1, depth=6, iterations=500, silent=True)

# Define the parameters for the search
param_grid_catboost = {
    'iterations': [100, 200, 300],  # Number of iterations (trees)
    'depth': [4, 6, 8],  # Depth of the trees
    'learning_rate': [0.01, 0.1, 0.2],   # Learning rate
    'l2_leaf_reg': [1, 3, 5], # L2 regularization
    'subsample': [0.7, 0.8, 0.9], # Fraction of samples used to fit each tree
    'auto_class_weights': [None, 'Balanced']  # Handling of class imbalance
}

# Set up the GridSearchCV
grid_search_catboost = GridSearchCV(estimator=catboost_model, param_grid=param_grid_catboost, cv=5, n_jobs=-1, verbose=2)

# Fit the model to the training data
grid_search_catboost.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:")
print(grid_search_catboost.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1215 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1215 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5220, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit

Mejores parámetros encontrados:
{'auto_class_weights': 'Balanced', 'depth': 8, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'subsample': 0.7}


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Get the best model
best_catboost = grid_search_catboost.best_estimator_

# Predict on the test set
y_pred_catboost = best_catboost.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_catboost))
print("Classification Report:\n", classification_report(y_test, y_pred_catboost))

Accuracy: 0.8877551020408163
Classification Report:
               precision    recall  f1-score   support

          No       0.89      0.99      0.94       255
         Yes       0.75      0.23      0.35        39

    accuracy                           0.89       294
   macro avg       0.82      0.61      0.65       294
weighted avg       0.87      0.89      0.86       294



#### So far, the best result is obtained from the hyperparameterized logistic regression, but we are encountering the problem of class imbalance. The model currently predicts employees who do not experience attrition better than those who do.